In [69]:
from pymongo.mongo_client import MongoClient
import yfinance as yf
import os
import httpx
import json
from dotenv import load_dotenv
load_dotenv()

True

In [70]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
client = MongoClient(os.environ["ATLAS_URI"])

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [8]:
print(client.list_database_names())

['sample_mflix', 'admin', 'local']


In [9]:
db = client["fomo_factory"]

Collection(Database(MongoClient(host=['ac-9peyuts-shard-00-01.tfgxdt6.mongodb.net:27017', 'ac-9peyuts-shard-00-02.tfgxdt6.mongodb.net:27017', 'ac-9peyuts-shard-00-00.tfgxdt6.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='stockCluster', authsource='admin', replicaset='atlas-emribc-shard-0', ssl=True), 'fomo_factory'), 'stock_prices')

In [66]:
db.drop_collection("stock_prices")
db.create_collection(
    "stock_prices",
    timeseries={
        "timeField": "timestamp",
        "metaField": "metadata",
        "granularity": "minutes"
    },
    expireAfterSeconds=60
)

Collection(Database(MongoClient(host=['ac-9peyuts-shard-00-01.tfgxdt6.mongodb.net:27017', 'ac-9peyuts-shard-00-02.tfgxdt6.mongodb.net:27017', 'ac-9peyuts-shard-00-00.tfgxdt6.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='stockCluster', authsource='admin', replicaset='atlas-emribc-shard-0', ssl=True), 'fomo_factory'), 'stock_prices')

In [33]:
stock_data_aapl=yf.Ticker("AAPL").history(period="5d", interval="1m")
stock_data_msft=yf.Ticker("MSFT").history(period="5d", interval="1m")


In [38]:
time_series_array=[]
for index, row in stock_data_msft.iterrows():
    time_series_array.append({
        "timestamp": index,
        "metadata": {
            "symbol": "AAPL",
        },
        "price":row["Close"],
    })

for index, row in stock_data_aapl.iterrows():
    time_series_array.append({
        "timestamp": index,
        "metadata": {
            "symbol": "MSFT",
        },
        "price":row["Close"],
    })

In [67]:
db["stock_prices"].insert_many(time_series_array)

In [36]:
stock_data_aapl

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-07-08 09:30:00-04:00,227.089996,227.350006,226.470001,226.922501,2348656,0.0,0.0
2024-07-08 09:31:00-04:00,226.949997,227.149994,226.550003,226.580002,395110,0.0,0.0
2024-07-08 09:32:00-04:00,226.559998,226.970001,226.539993,226.880005,282368,0.0,0.0
2024-07-08 09:33:00-04:00,226.853394,226.979996,226.490005,226.639999,344988,0.0,0.0
2024-07-08 09:34:00-04:00,226.645004,226.880005,226.619995,226.839996,317436,0.0,0.0
...,...,...,...,...,...,...,...
2024-07-12 15:55:00-04:00,231.020004,231.026993,230.389999,230.529999,254676,0.0,0.0
2024-07-12 15:56:00-04:00,230.529999,230.679993,230.500000,230.529999,159098,0.0,0.0
2024-07-12 15:57:00-04:00,230.539902,230.544998,230.360001,230.375000,166645,0.0,0.0


In [82]:
cursor=db["stock_prices"].find(
    {
        "metadata.symbol": "AAPL",
        "timestamp": {
            "$gte": stock_data_aapl.index[0],
            "$lte": stock_data_aapl.index[10]
        }
    },
    {
        "timestamp": 1,
        "price": 1,
        "_id": 0
    }
)

In [83]:
results = list(cursor)
for document in results:
    print(document)

In [71]:


url = "https://api.livecoinwatch.com/coins/map"

payload = json.dumps({
  "codes": ["ETH","BTC","GRIN"],
  "currency": "USD",
  "sort": "rank",
  "order": "ascending",
  "offset": 0,
  "limit": 0,
  "meta": False
})
headers = {
  'content-type': 'application/json',
  'x-api-key': os.environ["COIN_API_KEY"]
}

response = httpx.request("POST", url, headers=headers, data=payload)

print(response.text)

[{"code":"BTC","rate":59803.667874739076,"volume":17910507712,"cap":1179345494142,"delta":{"hour":1.0009,"day":1.0205,"week":1.0512,"month":0.8905,"quarter":0.9038,"year":1.9688}},{"code":"ETH","rate":3187.90659579325,"volume":8130075845,"cap":376274105076,"delta":{"hour":0.9969,"day":1.0121,"week":1.0731,"month":0.9095,"quarter":0.9928,"year":1.6442}},{"code":"GRIN","rate":0.02905128003626805,"volume":45086,"cap":5021417,"delta":{"hour":1.0075,"day":1.1366,"week":1.0701,"month":0.8771,"quarter":0.5709,"year":0.6524}}]


In [73]:
data=response.json()

In [74]:
data

[{'code': 'BTC',
  'rate': 59803.667874739076,
  'volume': 17910507712,
  'cap': 1179345494142,
  'delta': {'hour': 1.0009,
   'day': 1.0205,
   'week': 1.0512,
   'month': 0.8905,
   'quarter': 0.9038,
   'year': 1.9688}},
 {'code': 'ETH',
  'rate': 3187.90659579325,
  'volume': 8130075845,
  'cap': 376274105076,
  'delta': {'hour': 0.9969,
   'day': 1.0121,
   'week': 1.0731,
   'month': 0.9095,
   'quarter': 0.9928,
   'year': 1.6442}},
 {'code': 'GRIN',
  'rate': 0.02905128003626805,
  'volume': 45086,
  'cap': 5021417,
  'delta': {'hour': 1.0075,
   'day': 1.1366,
   'week': 1.0701,
   'month': 0.8771,
   'quarter': 0.5709,
   'year': 0.6524}}]

In [78]:
from datetime import datetime

'2024-07-14T20:05:39.825417'

In [80]:
time_series_array_v2=[]
for coin in data:
    time_series_array_v2.append({
        "timestamp": datetime.now().isoformat(),
        "metadata": {
            "symbol": coin["code"],
        },
        "price":coin["rate"]
    })
    

In [81]:
# get current timestamp 
time_series_array_v2

[{'timestamp': '2024-07-14T20:05:54.877260',
  'metadata': {'symbol': 'BTC'},
  'price': 59803.667874739076},
 {'timestamp': '2024-07-14T20:05:54.877270',
  'metadata': {'symbol': 'ETH'},
  'price': 3187.90659579325},
 {'timestamp': '2024-07-14T20:05:54.877274',
  'metadata': {'symbol': 'GRIN'},
  'price': 0.02905128003626805}]

In [84]:
cursor=db["stock_prices"].find({})

In [85]:
results = list(cursor)
for document in results:
    print(document)

{'timestamp': datetime.datetime(2024, 7, 14, 15, 36, 42, 61000), '_id': ObjectId('6693f08a59c8c34b01d62b8c'), 'price': 60131.47631073851, 'metadata': {'symbol': 'BTC'}}
{'timestamp': datetime.datetime(2024, 7, 14, 15, 36, 42, 61000), '_id': ObjectId('6693f08a59c8c34b01d62b8d'), 'price': 3191.4545184236276, 'metadata': {'symbol': 'ETH'}}
{'timestamp': datetime.datetime(2024, 7, 14, 15, 36, 42, 61000), '_id': ObjectId('6693f08a59c8c34b01d62b8e'), 'price': 0.028816165184506994, 'metadata': {'symbol': 'GRIN'}}
{'timestamp': datetime.datetime(2024, 7, 14, 15, 37, 1, 216000), '_id': ObjectId('6693f09d59c8c34b01d62b8f'), 'price': 60126.82787838685, 'metadata': {'symbol': 'BTC'}}
{'timestamp': datetime.datetime(2024, 7, 14, 15, 37, 1, 216000), '_id': ObjectId('6693f09d59c8c34b01d62b90'), 'price': 3191.3357533439103, 'metadata': {'symbol': 'ETH'}}
{'timestamp': datetime.datetime(2024, 7, 14, 15, 37, 1, 216000), '_id': ObjectId('6693f09d59c8c34b01d62b91'), 'price': 0.0288149017536504, 'metadata'